# Fill the shared array with data

In [1]:
import numpy as np
import skmap_bindings
import time
import tempfile
from pathlib import Path

def _model_input(tile, start_year = 2000, end_year = 2022, bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal'], base_url='http://192.168.49.30:8333'):
    prediction_layers = []

    for year in range(start_year, end_year + 1):
        for band in bands:
            prediction_layers += [
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0101_{year}0228_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0301_{year}0430_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0501_{year}0630_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0701_{year}0831_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0901_{year}1031_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}1101_{year}1231_go_epsg.4326_v20230908.tif'
            ]

    return prediction_layers

def _find_connected_indices(raster_files, str1, str2):
    list1 = []
    list2 = []    
    for i, elem1 in enumerate(raster_files):
        if str1 in elem1:
            for j, elem2 in enumerate(raster_files):
                if str2 in elem2 and elem1.split(str1)[-1] == elem2.split(str2)[-1]:
                    list1.append(i)
                    list2.append(j)
                    break                
    return list1, list2


In [2]:
raster_files = _model_input("013E_61N", start_year = 1997, end_year = 1998, bands = ['red', 'nir'], base_url='http://192.168.1.30:8333')
hosts = [ f'192.168.1.{i}:8333' for i in range(30,43) ]
raster_files = [str(r).replace("192.168.1.30", f"192.168.1.{30 + int.from_bytes(Path(r).stem.encode(), 'little') % len(hosts)}") for r in raster_files]
list_red, list_nir = _find_connected_indices(raster_files, 'red', 'nir')

n_feat = len(raster_files) + len(list_red)
n_threads = 48

In [3]:
x_off = 2
y_off = 2
x_size = 4000
y_size = 4000

bands_list = [1,] # The first band is indexed as 1

conf_GDAL = {"CPL_VSIL_CURL_ALLOWED_EXTENSIONS": ".tif",
             "GDAL_HTTP_MULTIRANGE": "SINGLE_GET",
             "GDAL_HTTP_CONNECTTIMEOUT": "320",
             "GDAL_HTTP_TIMEOUT": "320",
             "CPL_VSIL_CURL_USE_HEAD": "NO",
             "GDAL_HTTP_VERSION": "1.0",
             "CPL_CURL_GZIP": "NO"}

perm_vec = range(len(raster_files)) # row indices of the array where each tiff contenct will go (starting from first element of file_urls)
data = np.empty((n_feat,x_size*y_size), dtype=np.float32, order='C')
print("Starting")
start = time.time()
skmap_bindings.readData(data, n_threads, raster_files, perm_vec, x_off, y_off, x_size, y_size, bands_list, conf_GDAL, 255., np.nan)
print(f"C++ reading done in {time.time() - start:.2f} s")
print(data)
print(data.shape)

Starting
C++ reading done in 1.08 s
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan 10.]
 [15. 14. 14. ... 12. 12. 11.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
(36, 16000000)


In [4]:
print("Starting")
start = time.time()
skmap_bindings.computeNormalizedDifference(data, n_threads,
                            list_nir, list_red, range(len(raster_files), n_feat),
                            0.004, 0.004, 125., 125.)
print(f"C++ ND done in {time.time() - start:.2f} s")
print(data)


Starting
C++ ND done in 0.29 s
[[ nan  nan  nan ...  nan  nan  nan]
 [ nan  nan  nan ...  nan  nan  10.]
 [ 15.  14.  14. ...  12.  12.  11.]
 ...
 [214. 217. 217. ... 209. 209. 213.]
 [214. 217. 217. ... 210. 210. 211.]
 [214. 217. 217. ... 210. 210. 211.]]
